In [1]:
%load_ext autoreload
%autoreload 2
from qiskit_metal import draw, Dict, designs, MetalGUI
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
import qiskit_metal as metal
from just_claw import TransmonClaw
# ?metal.qlibrary.core.QComponent.add_qgeometry

design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

# dir(QComponent)
cpw_width = '11.7um'

design._chips.main.size.size_x = '5mm'
design._chips.main.size.size_y = '5mm'
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from collections import OrderedDict

import numpy as np

xmon1_options = dict(
    connection_pads = dict(
        readout = dict(
            connector_location='90',
            connector_type='0',
            claw_length="150um",
            ground_spacing='4.1um',
            claw_gap='5.1um',
            claw_width='15um',
            claw_cpw_length="0um",
            claw_cpw_width="10um"
        ),
    ),
    cross_width='30um',
    cross_length="100um",
    cross_gap='30um',
    orientation='-90',
    pos_x='-1500um',
    pos_y='1200um',
    # aedt_hfss_inductance=inductance,
    # hfss_inductance=inductance, 
    # q3d_inductance=inductance,
    # aedt_q3d_inductance=inductance
    hfss_inductance = 9.686e-09,
    hfss_capacitance = 0,
    hfss_resistance = 0,
    hfss_mesh_kw_jj = 7e-06,
    q3d_inductance = "10nH",
    q3d_capacitance = 0,
    q3d_resistance = 0,
    q3d_mesh_kw_jj = 7e-06,
    aedt_q3d_inductance = 1e-08,
    aedt_q3d_capacitance = 0,
    aedt_hfss_inductance = 9.686e-09,
    aedt_hfss_capacitance = 0
)

# Create a new Transmon Cross object with name 'Q1'
# q = TransmonClaw(design, 'Q', options=xmon1_options)

clt1_opts = Dict(prime_width = cpw_width,
                prime_gap = '5.1um',
                second_width = cpw_width,
                second_gap = '5.1um',
                coupling_space = '7.9um',
                coupling_length = '225um',
                open_termination = False,
                orientation = '-90',
                pos_y = '1200um',
                down_length = '50um')

# clt = CoupledLineTee(design, 'clt', clt1_opts)

cpw1_opts = Dict(pin_inputs = Dict(start_pin = Dict(component = 'clt',
                                                    pin = 'second_end'),
                                   end_pin = Dict(component = 'Q',
                                                  pin = 'readout')),
                # lead = Dict(start_straight = '50um',
                #             # start_jogged_extension = jogsS),
                # ),
                fillet = '49.9um',
                total_length = '3885.5930858724796um',#'3893.8166um', # '6776um',
                trace_width = cpw_width,
                # meander = Dict(spacing = '100um',
                #                asymmetry = '-115um'),
                trace_gap = '5.1um',)
# cpw = RouteMeander(design, 'cpw', options = cpw1_opts)

# gui.rebuild()
# gui.autoscale()
# gui.zoom_on_components(['clt5', 'Q5'])
# gui.screenshot()

In [2]:
import pandas as pd

In [3]:
aggregate_df = pd.read_csv("aggregate_df_1.csv")
aggregate_df

,updated_cross_lengths,updated_claw_lengths,updated_resonator_lengths,updated_coupling_dimensions,LJs,indices
0,213.641467,191.013989,3638.606336,236.406312,9.999932,"(7830, 1014)"
1,181.976585,145.265973,4587.817766,368.072758,10.000181,"(8615, 5533)"
2,181.004312,134.799356,4146.703745,92.527234,10.000019,"(3485, 2282)"
3,190.713307,134.451299,2990.119838,197.951335,10.000162,"(3465, 110)"
4,199.498695,134.246074,3582.612956,495.230165,10.000235,"(3415, 7756)"


In [4]:
Lj_vals = aggregate_df["LJs"].tolist()
predicted_cross_length = aggregate_df["updated_cross_lengths"].tolist()
predicted_claw_length = aggregate_df["updated_claw_lengths"].tolist()
cpw_lengths = aggregate_df["updated_resonator_lengths"].tolist()
cplr_lengths = aggregate_df["updated_coupling_dimensions"].tolist()

In [5]:
# Lj_vals = [14.82977874,12.76661344,11.5666809,14.02940303,9.329626635]
# predicted_cross_length = [172.2144875,201.49286,223.1790728,182.8509009,274.2789866]
# predicted_claw_length = [122.6728936,168.3926037,200.9612748,176.5367286,55.67607171]
# cpw_lengths = [4022.333844,3833.908348,3645.492252,4216.087604,4021.494855]
# cplr_lengths = [232.5421308,238.0444714,230.2712219,206.6035852,218.3920977]

In [6]:
predicted_cross_length

[213.641467334455,
 181.97658471821416,
 181.00431249856769,
 190.71330704069112,
 199.4986952317853]

In [7]:
ground_spacing = 4.1

In [8]:
comps_available = {
    "Sabrina":0,
    "Glinda":1,
    "Saruman":2,
    "VM1":3, #?
    "VM2":4, #?
}

In [9]:
indexes = (0,1,2,3,4)
version = "agg1"
comp_id = "Sabrina"  # Replace with actual computer name

# Eigenmode Sim (CLT + CPW + Claw)

In [10]:
from qiskit_metal.analyses.quantization import EPRanalysis
import pyEPR as epr
from pyaedt import Hfss
from utils import *
from objects import *
from datetime import datetime
import pprint

In [11]:
eig_q12bus = EPRanalysis(design, "hfss")

In [12]:
%matplotlib inline
eig_q12bus = EPRanalysis(design, "hfss")
hfss = eig_q12bus.sim.renderer

for i, (cross_length, claw_length, Lj, cpw_len, cplr_len) in enumerate(zip(predicted_cross_length, predicted_claw_length, Lj_vals, cpw_lengths, cplr_lengths)):
    if i not in indexes:
        continue
    # print(i)
    xmon1_options["connection_pads"]["readout"]["claw_length"] = f"{claw_length}um"
    xmon1_options["connection_pads"]["readout"]["ground_spacing"] = f"{ground_spacing}um"
    xmon1_options["aedt_hfss_inductance"]=Lj,
    xmon1_options["hfss_inductance"]=Lj, 
    xmon1_options["q3d_inductance"]=Lj,
    xmon1_options["aedt_q3d_inductance"]=Lj

    cpw1_opts["total_length"] = f"{cpw_len}um"
    clt1_opts["coupling_length"] = f"{int(round(cplr_len, 0))}um"
    
    v2_opts = Dict(
        claw_opts = xmon1_options,
        cplr_opts = clt1_opts,
        cpw_opts = cpw1_opts
    )
    # pprint.pprint(v2_opts)
    now = datetime.now()
    #print(xmon1_options)
    #print(cpw1_opts)
    #print(clt1_opts)
    print()
    CLT_epr_sweep(design = design, sweep_opts=v2_opts, filename = f"cavity_claw_num{i}_{comp_id}_v{version}")


INFO 01:22PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:22PM [load_ansys_project]: 	Opened Ansys App
INFO 01:22PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 01:22PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/lfl/Documents/Ansoft/
	Project:   4p1_xmon_LOM_sweep



Starting the Simulation


INFO 01:23PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]
INFO 01:23PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:23PM [connect]: 	Connected to project "4p1_xmon_LOM_sweep" and design "CavitySweep_hfss" 😀 

INFO 01:23PM [connect_design]: 	Opened active design
	Design:    CavitySweep1 [Solution type: Eigenmode]
WARNING 01:23PM [connect_setup]: 	No design setup detected.
WARNING 01:23PM [connect_setup]: 	Creating eigenmode default setup.
INFO 01:23PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


INFO 01:23PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]


creaing porst


# LOM Analysis

In [ ]:
# xmon = TransmonCross(design = design, options = xmon1_options, name = "xmon")

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [ ]:
c1.sim.setup.reuse_selected_design = False
c1.sim.setup.reuse_setup = False

# example: update single setting
c1.sim.setup.max_passes = 30
c1.sim.setup.min_converged_passes = 1
c1.sim.setup.percent_error = 0.1
c1.sim.setup.name = 'sweep_setup'

In [ ]:
def do_LOM_qubitonly(cross_length, claw_length, Lj, i):
    qname = 'xmon'
    cname = 'readout'

    # design.components[qname].options['connection_pads']['readout']['claw_length'] = f'{claw_length}um'
    # design.components[qname].options['cross_length'] = f'{cross_length}um'
    # design.components[qname].options['hfss_inductance'] = Lj
    # design.components[qname].options['aedt_hfss_inductance'] = Lj
    # design.components[qname].options['q3d_inductance'] = Lj
    # design.components[qname].options['aedt_q3d_inductance'] = Lj
    # xmon1_options["cross_width"] = f"15um"
    xmon1_options["cross_length"] = f"{cross_length}um"
    xmon1_options["connection_pads"]["readout"]["claw_length"] = f"{claw_length}um"
    # xmon1_options["connection_pads"]["readout"]["ground_spacing"] = f"{ground_spacing}um"
    xmon1_options["aedt_hfss_inductance"]=Lj,
    xmon1_options["hfss_inductance"]=Lj, 
    xmon1_options["q3d_inductance"]=Lj,
    xmon1_options["aedt_q3d_inductance"]=Lj
    q = TransmonCross(design, qname, options=xmon1_options)
    design.rebuild()
    selection = [qname]
    open_pins = [(qname, cname)]
    print(q.options)
    c1.sim.renderer.clean_active_design()
    c1.sim.run(name = 'LOMv2.0', components=selection,
               open_terminations=open_pins)
    cap_df = c1.sim.capacitance_matrix

    print(f'For Qubit_{i}, the cap matrix is...')
    print(cap_df)

    data = {
        "design_options": design.components[qname].options,
        "data": {
            "cross_to_ground": 0 if 'ground_main_plane' not in cap_df.loc[f'cross_{qname}'] else cap_df.loc[f'cross_{qname}']['ground_main_plane'],
            "claw_to_ground": 0 if 'ground_main_plane' not in cap_df.loc[f'{cname}_connector_arm_{qname}'] else cap_df.loc[f'{cname}_connector_arm_{qname}']['ground_main_plane'],
            "cross_to_claw": cap_df.loc[f'cross_{qname}'][f'{cname}_connector_arm_{qname}'],
            "cross_to_cross": cap_df.loc[f'cross_{qname}'][f'cross_{qname}'],
            "claw_to_claw": cap_df.loc[f'{cname}_connector_arm_{qname}'][f'{cname}_connector_arm_{qname}'],
            "ground_to_ground": 0 if 'ground_main_plane' not in cap_df.loc[f'cross_{qname}'] else cap_df.loc['ground_main_plane']['ground_main_plane']
        },
        "sim_info": {
            "setup": c1.sim.setup,
            "renderer_options": c1.sim.renderer.options
        }
    }
    save_simulation_data_to_json(data, filename = f"qubitonly_num{i}_{comp_id}_v{version}")


In [ ]:
all_cap_matricies = []

for i, (cross_length, claw_length, Lj) in enumerate(zip(predicted_cross_length, predicted_claw_length, Lj_vals)):
    # print(claw_length)
    if i not in indexes:
        continue
    do_LOM_qubitonly(cross_length, claw_length, Lj, i)

In [ ]:
closest_df = pd.read_csv("closest_df_dday-1b.csv", index_col=[0])
closest_df

,alpha,f_res,kappa,cross_to_ground,cross_to_claw,cross_to_cross,claw_to_claw,ground_to_ground,claw_to_ground,cross_length_lom,claw_length,coupling_length,total_length,uid,alpha_ratio,g_ratio
g,,,,,,,,,,,,,,,,
130.132405,-144.078740,6.716950e+09,653064.894380,-136.72515,-9.05640,-136.72515,124.88586,310.76713,-115.21454,280.0,250.0,500.0,3400.0,system_0,1.021217,1.030713
111.847293,-222.231151,9.198423e+09,267594.674642,-91.21653,-5.38292,-91.21653,87.87879,223.72845,-81.98676,190.0,150.0,200.0,2700.0,system_1,1.022043,1.033596
129.894869,-290.844156,6.700301e+09,144246.490912,-69.95780,-5.10870,-69.95780,80.53424,191.56325,-74.92429,150.0,130.0,230.0,3900.0,system_2,0.970690,0.949119
85.741762,-210.048249,5.353550e+09,161106.598429,-96.15117,-5.72667,-96.15117,91.56056,232.64043,-85.27078,200.0,160.0,350.0,4700.0,system_3,1.025230,0.992605
21.996392,-144.045422,9.441389e+09,205508.755750,-143.52229,-2.29173,-143.52229,58.45243,258.69919,-55.90335,280.0,70.0,200.0,2700.0,system_4,1.055893,0.921644


In [ ]:
cross_lengths = closest_df["cross_length_lom"]
claw_lengths = closest_df["claw_length"]

In [ ]:
all_cap_matricies = []

for i, (cross_length, claw_length, Lj) in enumerate(zip(cross_lengths, claw_lengths, Lj_vals)):
    # print(claw_length)
    if i not in indexes:
        continue
    do_LOM_qubitonly(cross_length, claw_length, Lj, i)

{'pos_x': '-1500um', 'pos_y': '1200um', 'orientation': '-90', 'chip': 'main', 'layer': '1', 'connection_pads': {'readout': {'connector_type': '0', 'claw_length': '250.0um', 'ground_spacing': '10um', 'claw_width': '15um', 'claw_gap': '5.1um', 'claw_cpw_length': '40um', 'claw_cpw_width': '10um', 'connector_location': '90'}}, 'cross_width': '30um', 'cross_length': '280.0um', 'cross_gap': '30um', 'hfss_inductance': (10.00003130319056,), 'hfss_capacitance': 0, 'hfss_resistance': 0, 'hfss_mesh_kw_jj': 7e-06, 'q3d_inductance': (10.00003130319056,), 'q3d_capacitance': 0, 'q3d_resistance': 0, 'q3d_mesh_kw_jj': 7e-06, 'gds_cell_name': 'my_other_junction', 'aedt_q3d_inductance': 10.00003130319056, 'aedt_q3d_capacitance': 0, 'aedt_hfss_inductance': (10.00003130319056,), 'aedt_hfss_capacitance': 0}


INFO 01:47PM [connect_design]: 	Opened active design
	Design:    LOMv2.0_q3d18 [Solution type: Q3D]
WARNING 01:47PM [connect_setup]: 	No design setup detected.
WARNING 01:47PM [connect_setup]: 	Creating Q3D default setup.
INFO 01:47PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:47PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:47PM [analyze]: Analyzing setup sweep_setup


<COMObject GetModule>


INFO 01:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpbfkkcpsa.txt, C, , sweep_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 01:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpqsnjr80o.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 01:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp1cpc7dvs.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 01:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp_ke4nkc5.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 01:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpnwudnpu0.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell,

For Qubit_0, the cap matrix is...
                            cross_xmon  ground_main_plane  \
cross_xmon                   147.96257         -139.02998   
ground_main_plane           -139.02998          319.68939   
readout_connector_arm_xmon    -6.58646         -121.76263   

                            readout_connector_arm_xmon  
cross_xmon                                    -6.58646  
ground_main_plane                           -121.76263  
readout_connector_arm_xmon                   128.92945  
{'pos_x': '-1500um', 'pos_y': '1200um', 'orientation': '-90', 'chip': 'main', 'layer': '1', 'connection_pads': {'readout': {'connector_type': '0', 'claw_length': '150.0um', 'ground_spacing': '10um', 'claw_width': '15um', 'claw_gap': '5.1um', 'claw_cpw_length': '40um', 'claw_cpw_width': '10um', 'connector_location': '90'}}, 'cross_width': '30um', 'cross_length': '190.0um', 'cross_gap': '30um', 'hfss_inductance': (10.000322969703006,), 'hfss_capacitance': 0, 'hfss_resistance': 0, 'hfss_mesh

...

For Qubit_3, the cap matrix is...
                            cross_xmon  ground_main_plane  \
cross_xmon                   104.35588          -98.17143   
ground_main_plane            -98.17143          240.59289   
readout_connector_arm_xmon    -4.13180          -91.00301   

                            readout_connector_arm_xmon  
cross_xmon                                    -4.13180  
ground_main_plane                            -91.00301  
readout_connector_arm_xmon                    95.65249  
{'pos_x': '-1500um', 'pos_y': '1200um', 'orientation': '-90', 'chip': 'main', 'layer': '1', 'connection_pads': {'readout': {'connector_type': '0', 'claw_length': '70.0um', 'ground_spacing': '10um', 'claw_width': '15um', 'claw_gap': '5.1um', 'claw_cpw_length': '40um', 'claw_cpw_width': '10um', 'connector_location': '90'}}, 'cross_width': '30um', 'cross_length': '280.0um', 'cross_gap': '30um', 'hfss_inductance': (10.000399033209996,), 'hfss_capacitance': 0, 'hfss_resistance': 0, 'hfss_mesh_

INFO 01:59PM [connect_design]: 	Opened active design
	Design:    LOMv2.0_q3d22 [Solution type: Q3D]
WARNING 01:59PM [connect_setup]: 	No design setup detected.
WARNING 01:59PM [connect_setup]: 	Creating Q3D default setup.
INFO 01:59PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:59PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:59PM [analyze]: Analyzing setup sweep_setup


<COMObject GetModule>


INFO 02:01PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpo2yiu0w3.txt, C, , sweep_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:01PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp492ajc6x.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:01PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpwx_k4n58.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 02:01PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpazz52bt0.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 02:01PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpx_r77c59.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell,

For Qubit_4, the cap matrix is...
                            cross_xmon  ground_main_plane  \
cross_xmon                   147.90618         -143.92242   
ground_main_plane           -143.92242          262.62816   
readout_connector_arm_xmon    -1.64726          -59.39525   

                            readout_connector_arm_xmon  
cross_xmon                                    -1.64726  
ground_main_plane                            -59.39525  
readout_connector_arm_xmon                    61.33336  


In [ ]:
print("test")

test
